In [1]:
import xarray as xr
import numpy as np
import numpy as np
import xarray as xr
import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

In [30]:
import os
import xarray as xr
import pandas as pd


def open_with_cftime(file_path, engine="netcdf4"):
    ds = xr.open_dataset(
        file_path,
        engine=engine,
        decode_times=True,
        use_cftime=True
    )
    iso_times = ds['time'].dt.strftime('%Y-%m-%dT%H:%M:%S').values
    pd_times = pd.to_datetime(iso_times)
    return ds.assign_coords(time=pd_times)


def compute_weekly_climatology(file_list, variable=None):
    datasets = [open_with_cftime(fp) for fp in file_list if os.path.isfile(fp)]
    combined = xr.concat(datasets, dim='time', data_vars='minimal', coords='minimal')
    da = combined[variable] if variable else combined
    daily = da.resample(time='1D').sum(dim='time')
    daily = daily.assign_coords(weekofyear=daily['time'].dt.isocalendar().week)
    weekly_clim = daily.groupby('weekofyear').mean(dim='time')
    return weekly_clim


def compute_daily_series(file_path, variable=None):
    ds = open_with_cftime(file_path)
    da = ds[variable] if variable else ds
    daily = da.resample(time='1D').sum(dim='time')
    return daily


def detect_droughts(daily, weekly_clim, min_run=2):
    """
    Detect drought events: runs of at least `min_run` consecutive days where daily < threshold,
    with threshold defined as 10% of the weekly mean daily energy.
    Returns:
      event_count    = number of events per grid cell
      mean_duration  = mean duration of events per grid cell
    """
    weeks = daily['time'].dt.isocalendar().week
    threshold = 0.1 * weekly_clim.sel(weekofyear=weeks)
    drought = daily < threshold

    template = drought.isel(time=0)
    event_count = xr.zeros_like(template)
    current = xr.zeros_like(template)
    durations = []

    nt = drought.sizes['time']
    for t in range(nt):
        mask = drought.isel(time=t)
        current = xr.where(mask, current + 1, 0)
        new_event = (current == min_run)
        event_count = event_count + new_event

        if t < nt - 1:
            next_mask = drought.isel(time=t+1)
            end_run = (~next_mask) & (current >= min_run)
        else:
            end_run = (current >= min_run)
        run_lengths = current.where(end_run)
        durations.append(run_lengths)

    if durations:
        # concatenate durations with override to ignore coordinate mismatches
        all_durs = xr.concat(durations, dim='event', compat='override')
        mean_duration = all_durs.mean(dim='event')
    else:
        mean_duration = xr.zeros_like(template)

    return event_count, mean_duration



In [24]:
# --- User parameters: adjust as needed ---
diri = "/work/users/s233224/Climate-Change-Impacted-Solar-Energy-Generation/power/"
models = [
    "ACCESS-CM2", "CanESM5", "CMCC-CM2-SR5", "CMCC-ESM2",
    "HadGEM3-GC31-LL", "HadGEM3-GC31-MM", "MRI-ESM2-0"
]
variable_name = "specific generation"

# Define periods and years
periods = {
    "BOC": {"subpath": "historical", "years": list(range(1980, 2015))},
    "EOC": {"subpath": "ssp585",    "years": list(range(2065, 2100))}
}

# Container for storing results
drought_stats = {}

for model in models:
    drought_stats[model] = {}
    for period, info in periods.items():
        base_dir = os.path.join(diri, f"{model}_1x1grid", info['subpath'])
        # Prepare file list for weekly climatology
        files = [os.path.join(base_dir, f"solar_power_{yr}.nc") for yr in info['years']]
        weekly_clim = compute_weekly_climatology(files, variable=variable_name)


/tmp/ipykernel_94926/2725828434.py:7: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  ds = xr.open_dataset(
/tmp/ipykernel_94926/2725828434.py:7: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  ds = xr.open_dataset(
/tmp/ipykernel_94926/2725828434.py:7: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(

In [31]:

droughts_agg = {}
for period, info in periods.items():
    all_counts = []
    all_meandurs = []
    for model in models:
        base_dir = os.path.join(diri, f"{model}_1x1grid", info['subpath'])
        files = [os.path.join(base_dir, f"solar_power_{yr}.nc") for yr in info['years']]
        #weekly_clim = compute_weekly_climatology(files, variable=variable_name)
        for yr in info['years']:
            year_file = os.path.join(base_dir, f"solar_power_{yr}.nc")
            if not os.path.isfile(year_file):
                continue
            daily = compute_daily_series(year_file, variable=variable_name)
            cnt, mdur = detect_droughts(daily, weekly_clim, min_run=2)
            all_counts.append(cnt)
            all_meandurs.append(mdur)
    # after looping all models and years for this period
    counts_stack = xr.concat(all_counts, dim='model_year', compat='override')
    mdurs_stack = xr.concat(all_meandurs, dim='model_year', compat='override')
    droughts_agg[period] = {
        'mean_event_count': counts_stack.mean(dim='model_year'),
        'mean_event_duration': mdurs_stack.mean(dim='model_year')
    }

# Results accessible as:
# droughts_agg['BOC']['mean_event_count']
# droughts_agg['BOC']['mean_event_duration']

/tmp/ipykernel_94926/3580825923.py:7: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  ds = xr.open_dataset(


ValueError: Cannot specify both coords='different' and compat='override'.